In [1]:
import pandas as pd
import altair as alt
import requests
import numpy as np
import os
import eco_style
alt.themes.enable("light")

ThemeRegistry.enable('light')

In [2]:
prices_df = pd.read_csv('/Users/finn/Dropbox/2. Data Science/DataScience2024_LSE/Week8_BigData/seminar/UKMicroData/db_prices.csv')
items_df = pd.read_csv('/Users/finn/Dropbox/2. Data Science/DataScience2024_LSE/Week8_BigData/seminar/UKMicroData/db_item_clean.csv')
full_df = pd.merge(prices_df, items_df, on='item_id')

In [3]:
def get_hsl(hsl_str):
    h, s, l = hsl_str.replace('hsl(', '').replace(')', '').replace('%', '').replace(" ", "").split(',')
    return int(h), int(s), int(l)

def interpolate(start, end, steps):
    return [start + (end - start) * i / (steps - 1) for i in range(steps)]

def create_symmetric_hsl(low, peak, stops):
    low_h, low_s, low_l = get_hsl(low)
    peak_h, peak_s, peak_l = get_hsl(peak)

    # Interpolate upwards (half the list, excluding the peak)
    h_values = interpolate(low_h, peak_h, stops // 2 + 1)
    s_values = interpolate(low_s, peak_s, stops // 2 + 1)
    l_values = interpolate(low_l, peak_l, stops // 2 + 1)

    # Combine into HSL strings (up to peak, then mirrored downwards)
    hsl_up = [f"hsl({int(h)}, {int(s)}%, {int(l)}%)" for h, s, l in zip(h_values, s_values, l_values)]
    hsl_full = hsl_up + hsl_up[-2::-1]  # Mirror excluding the peak to form the full 9 stops
    
    return hsl_full

# Input HSL strings
peak = "hsl(15, 100%, 57%)"
low = "hsl(0, 0%, 88%)"

# Generate 9 stops with peak in the middle
hsl_colors = ["transparent"]+create_symmetric_hsl(low, peak, 9)

# Output the result
for color in hsl_colors:
    print(color)

transparent
hsl(0, 0%, 88%)
hsl(3, 25%, 80%)
hsl(7, 50%, 72%)
hsl(11, 75%, 64%)
hsl(15, 100%, 57%)
hsl(11, 75%, 64%)
hsl(7, 50%, 72%)
hsl(3, 25%, 80%)
hsl(0, 0%, 88%)


In [4]:
full_df

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id,description,date_quote_s,date_quote_e,n_obs
0,199108.0,2.0,210101,2.0,0.49000,NaN,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
1,199007.0,10.0,210101,11.0,0.52000,NaN,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
2,199007.0,2.0,210101,12.0,0.37000,NaN,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
3,199811.0,179.0,210101,8.0,0.69000,NaN,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
4,199108.0,1.0,210101,10.0,0.47000,NaN,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
...,...,...,...,...,...,...,...,...,...,...,...
46689338,200704.0,441.0,640406,4.0,55.00000,NaN,640406,HOTEL 1 NIGHT PRICE,200502,202409,26516
46689339,200701.0,731.0,640406,3.0,85.00000,NaN,640406,HOTEL 1 NIGHT PRICE,200502,202409,26516
46689340,201210.0,5074.0,640406,3.0,40.00000,NaN,640406,HOTEL 1 NIGHT PRICE,200502,202409,26516
46689341,200812.0,542.0,640406,2.0,45.00000,NaN,640406,HOTEL 1 NIGHT PRICE,200502,202409,26516


In [5]:
items_df[items_df.description.astype(str).str.contains('chicken', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
75,210903,FROZEN CHICKEN PIECES,198802,200301,23479
77,210905,FRESH/CHILLED CHICKEN PER KG,198903,202409,102374
78,210906,FRZEN ROASTING CHICKEN PER KG,198903,200601,27761
79,210907,FRESH/CHILLED CHICKEN PIECES,199502,200301,25596
81,210909,FROZEN CHICKEN THIGHS PER KG,200302,200601,2156
82,210910,FRESH BONELESS CHICKEN BREAST,200302,202409,62018
84,210912,FROZEN CHICKEN BREASTS,200602,202001,28902
85,210913,ROTISSERIE CHICKEN [WHOLE],200902,202401,8500
103,211019,FROZ CHICKEN NUGGETS 220-600G,200502,202409,26368
104,211021,CHICKEN KIEV 2 PACK 250-300G,200602,201501,7172


In [6]:
targets = [
    210905,
    "POTATOES- BAKING PR KG",
    310406
]



In [7]:
df = full_df[full_df['description'].isin(targets) | full_df['item_id'].isin(targets)]
df['quote_date'] = pd.to_datetime(df['quote_date'], format='%Y%m')
df = df[['item_id', 'description', 'price', 'quote_date']]
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# group by item_id and month and get the deciles
df = df.groupby(['item_id', 'description', 'quote_date']).quantile(quantiles).reset_index()

df = df.rename(columns={'level_3': 'quantile', 'price': 'price'})

proper_quantiles = df.copy()

df = df.pivot_table(index=['item_id', 'description', 'quote_date'], columns='quantile', values='price').reset_index()

cols = [0.1,	0.2,	0.3,	0.4,	0.5,	0.6,	0.7,0.8,	0.9]
for i in range(len(cols), 0,-1):
    print(cols[i-1])
    df[cols[i-1]] = df[cols[i-1]] - df[cols[i-2]] if i > 1 else df[cols[i-1]]

df = df.melt(id_vars=['item_id', 'description', 'quote_date'], value_vars=cols, var_name='quantile', value_name='price')
df = df.merge(proper_quantiles, on=['item_id', 'description', 'quote_date', 'quantile'], suffixes=('', '_true'))

0.9
0.8
0.7
0.6
0.5
0.4
0.3
0.2
0.1


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_5999/1341751562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quote_date'] = pd.to_datetime(df['quote_date'], format='%Y%m')


In [8]:
[["transparent"]+create_symmetric_hsl("hsl(0, 0%, 88%)", "hsl(15, 100%, 57%)", 9)]

[['transparent',
  'hsl(0, 0%, 88%)',
  'hsl(3, 25%, 80%)',
  'hsl(7, 50%, 72%)',
  'hsl(11, 75%, 64%)',
  'hsl(15, 100%, 57%)',
  'hsl(11, 75%, 64%)',
  'hsl(7, 50%, 72%)',
  'hsl(3, 25%, 80%)',
  'hsl(0, 0%, 88%)']]

In [13]:
temp_df = df.query("item_id == 310406")


peak = "hsl(15, 100%, 57%)"
low = "hsl(0, 0%, 88%)"

schemes = [
    ["transparent"]+create_symmetric_hsl("hsl(0, 0%, 88%)", "hsl(15, 100%, 57%)", 9),
    ["transparent"]+create_symmetric_hsl("hsl(0, 0%, 88%)", "hsl(45, 89%, 59%)", 9),
    ["transparent"]+create_symmetric_hsl("hsl(0, 0%, 88%)", "hsl(350, 70%, 46%)", 9)
]

charts = []

for i, item_id in enumerate(df.item_id.unique()):
    temp_df = df.query(f"item_id == {item_id}")

    print(schemes[i])

    quantiles = alt.Chart(temp_df).mark_area(
        interpolate='basis',
    ).encode(
        x='quote_date:T',
        y=alt.Y('price:Q', title="", axis=alt.Axis(labelExpr="'£'+datum.label")),
        color=alt.Color('quantile:O', 
                        legend=None,
                        scale=alt.Scale(
                            range=schemes[i]    
            )),
        tooltip=[
            {'type': 'quantitative', 'field': 'quantile', 'title': 'Quantile'},
            {'type': 'quantitative', 'field': 'price_true', 'title': 'Price'},
            {'type': 'temporal', 'field': 'quote_date', 'title': 'Date'},
            {'type': 'nominal', 'field': 'description', 'title': 'Item'}
        ],
        order=alt.Order('quantile', sort='ascending')
    ).properties(
        width=300,
        height=350
    )

    median_line = alt.Chart(temp_df.query("quantile == 0.5")).mark_line(color='white', strokeWidth=1).encode(
        x=alt.X('quote_date:T', title=''),
        y=alt.Y('price_true:Q', title='', axis=alt.Axis(labelExpr="'£'+datum.label")),
    )

    print(temp_df['description'].iloc[0].title())  

    title_rebinds = {
        "Fresh/Chilled Chicken  Per Kg": "Chicken (per kg)",
        "Fortified Wine  (70-75Cl)": "Fortified Wine (70-75cl)",
        "Potatoes- Baking Pr Kg": "Potatoes (per kg)"
    } 

    chart = (quantiles).properties(
        title=alt.TitleParams(
        #text=f"{temp_df['description'].iloc[0].title()}",
        text=title_rebinds[temp_df['description'].iloc[0].title()],
        fontWeight='normal',
        
        )
    )

    charts.append(chart)

chart = alt.hconcat(*charts)
chart.save("price_distribution.png", scale_factor=3)
chart.save("price_distribution.json")

chart.display()

['transparent', 'hsl(0, 0%, 88%)', 'hsl(3, 25%, 80%)', 'hsl(7, 50%, 72%)', 'hsl(11, 75%, 64%)', 'hsl(15, 100%, 57%)', 'hsl(11, 75%, 64%)', 'hsl(7, 50%, 72%)', 'hsl(3, 25%, 80%)', 'hsl(0, 0%, 88%)']
Fresh/Chilled Chicken  Per Kg
['transparent', 'hsl(0, 0%, 88%)', 'hsl(11, 22%, 80%)', 'hsl(22, 44%, 73%)', 'hsl(33, 66%, 66%)', 'hsl(45, 89%, 59%)', 'hsl(33, 66%, 66%)', 'hsl(22, 44%, 73%)', 'hsl(11, 22%, 80%)', 'hsl(0, 0%, 88%)']
Fortified Wine  (70-75Cl)
['transparent', 'hsl(0, 0%, 88%)', 'hsl(87, 17%, 77%)', 'hsl(175, 35%, 67%)', 'hsl(262, 52%, 56%)', 'hsl(350, 70%, 46%)', 'hsl(262, 52%, 56%)', 'hsl(175, 35%, 67%)', 'hsl(87, 17%, 77%)', 'hsl(0, 0%, 88%)']
Potatoes- Baking Pr Kg


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.HConcatChart(...)

## Adjust for income

In [10]:
base = "https://api.beta.ons.gov.uk/v1/data?uri="
uri = "/employmentandlabourmarket/peopleinwork/earningsandworkinghours/timeseries/kab9/emp"

url = base + uri

req = requests.get(url)
data = req.json()['months']
data

full_wage_df = pd.DataFrame(data)

In [11]:
df = full_wage_df.copy()
df['date'] = pd.to_datetime(df['date'], format='%Y %b')
df = df[['date', 'value']].rename(columns={'value': 'wage'})
wage_df = df.copy()

In [12]:
df = proper_quantiles.copy()
df = df.merge(wage_df, left_on='quote_date', right_on='date', how='inner')
df.description.unique()
df['wage'] = df['wage'].astype(float)
df['price'] = df['price'].astype(float)
df['adjusted_price'] = df['wage']/df['price']
df

df['adjusted_price'] = np.where(df.item_id == 212311, df['adjusted_price']*5, df['adjusted_price'])
df['adjusted_price'] = np.where(df.item_id == 310406, df['adjusted_price']*0.75, df['adjusted_price'])

df['label'] = np.where(df.quote_date == '2024-09-01', df['description'], '')

charts = []
#for item_id in [310406, 212311]:
for item_id in df.item_id.unique():

    temp_df = df.query(f"item_id == {item_id} and quantile == 0.5")

    labelExprs = {
        212311: "datum.label + ' potatoes/week'",
        310406: "datum.label + 'L port/week'",
        210905: "datum.label + 'KG chicken/week'"
    }

    base = alt.Chart(temp_df).mark_line(color='black').encode(
        x=alt.X('quote_date:T', title=''),
        y=alt.Y('adjusted_price:Q', title='', axis=alt.Axis(labelExpr=labelExprs[item_id])),
        color=alt.value('hsl(351, 100%, 57%)')
    )

    line = base.mark_line(color='black').encode(
        x=alt.X('quote_date:T', title=''),
    )

    end_label = base.mark_text(align='left', dx=5, dy=-5).encode(
        text='label:N'
    )

    title_rebinds = {
        "Fresh/Chilled Chicken  Per Kg": "Chicken (per kg)",
        "Fortified Wine  (70-75Cl)": "Fortified Wine (per bottle)",
        "Potatoes- Baking Pr Kg": "Potatoes"
    } 


    chart = (line ).properties(
        width=275,
        height=350,
        title=alt.TitleParams(
        #text=f"{temp_df['description'].iloc[0].title()}",
        text=title_rebinds[temp_df['description'].iloc[0].title()],
        fontWeight='normal',
        )
    )

    charts.append(chart)

chart = alt.hconcat(*charts)
chart.save("price_distribution_adjusted.png", scale_factor=3)
chart.save("price_distribution_adjusted.json")
chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.HConcatChart(...)

In [239]:
df.query("item_id == 310406 and quantile == 0.5 and quote_date>='2004-09-01'")

,item_id,description,quote_date,quantile,price,date,wage,adjusted_price,label
1093,310406,FORTIFIED WINE (70-75CL),2004-09-01,0.5,5.98,2004-09-01,369.0,46.279264,
1120,310406,FORTIFIED WINE (70-75CL),2004-10-01,0.5,5.99,2004-10-01,372.0,46.577631,
1147,310406,FORTIFIED WINE (70-75CL),2004-11-01,0.5,5.98,2004-11-01,372.0,46.655518,
1174,310406,FORTIFIED WINE (70-75CL),2004-12-01,0.5,5.49,2004-12-01,373.0,50.956286,
1201,310406,FORTIFIED WINE (70-75CL),2005-01-01,0.5,5.99,2005-01-01,379.0,47.454092,
...,...,...,...,...,...,...,...,...,...
7285,310406,FORTIFIED WINE (70-75CL),2024-05-01,0.5,12.50,2024-05-01,687.0,41.220000,
7312,310406,FORTIFIED WINE (70-75CL),2024-06-01,0.5,13.00,2024-06-01,689.0,39.750000,
7339,310406,FORTIFIED WINE (70-75CL),2024-07-01,0.5,13.25,2024-07-01,690.0,39.056604,
7366,310406,FORTIFIED WINE (70-75CL),2024-08-01,0.5,13.00,2024-08-01,694.0,40.038462,


In [219]:
df.drop_duplicates(subset=['item_id'])

,item_id,description,quote_date,quantile,price,date,wage,adjusted_price
0,210905,FRESH/CHILLED CHICKEN PER KG,2000-01-01,0.1,1.747,2000-01-01,305.0,174.585003
9,310406,FORTIFIED WINE (70-75CL),2000-01-01,0.1,3.590,2000-01-01,305.0,84.958220
891,212311,POTATOES- BAKING PR KG,2004-02-01,0.1,0.530,2004-02-01,352.0,664.150981


In [183]:
df = pd.read_excel("lms.xlsx")


/Users/finn/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [190]:
"Seasonally Adjusted Total Pay".lower()

'seasonally adjusted total pay'

In [191]:
cols = df.columns
[c for c in cols if 'seasonally adjusted total pay' in c.lower()]

['AWE: Whole Economy Real Terms Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay',
 'AWE: Whole Economy Real Terms Year on Year Three Month Growth (%): Seasonally Adjusted Total Pay',
 'AWE: Whole Economy Real Terms Level (£): Seasonally Adjusted Total Pay',
 'AWE: Services Level (£): Seasonally Adjusted Total Pay Excluding Arrears',
 'AWE: Services Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay Excluding Arrears',
 'AWE: Services Year on Year Three Month Average Growth (%): Seasonally Adjusted Total Pay Excluding Arrears',
 'AWE: Finance and Business Services Level (£): Seasonally Adjusted Total Pay Excluding Arrears',
 'AWE: Finance and Business Services Year on Year Single Month Growth (%): Seasonally Adjusted Total Pay Excluding Arrears',
 'AWE: Finance and Business Services Year on Year Three Month Average Growth (%): Seasonally Adjusted Total Pay Excluding Arrears',
 'AWE: Manufacturing Level (£): Seasonally Adjusted Total Pay Excluding Arr

In [165]:
schemes[0

['transparent',
 'hsl(0, 0%, 88%)',
 'hsl(3, 25%, 80%)',
 'hsl(7, 50%, 72%)',
 'hsl(11, 75%, 64%)',
 'hsl(15, 100%, 57%)',
 'hsl(11, 75%, 64%)',
 'hsl(7, 50%, 72%)',
 'hsl(3, 25%, 80%)',
 'hsl(0, 0%, 88%)']

In [182]:
df.query("item_id == 212311 and quantile == 0.5")

,item_id,description,quote_date,quantile,price,price_true
4795,212311,POTATOES- BAKING PR KG,2004-02-01,0.5,0.01,0.86
4796,212311,POTATOES- BAKING PR KG,2004-03-01,0.5,0.08,0.94
4797,212311,POTATOES- BAKING PR KG,2004-04-01,0.5,0.06,0.95
4798,212311,POTATOES- BAKING PR KG,2004-05-01,0.5,0.00,0.99
4799,212311,POTATOES- BAKING PR KG,2004-06-01,0.5,0.00,0.99
...,...,...,...,...,...,...
5018,212311,POTATOES- BAKING PR KG,2024-05-01,0.5,0.01,0.80
5019,212311,POTATOES- BAKING PR KG,2024-06-01,0.5,0.01,0.80
5020,212311,POTATOES- BAKING PR KG,2024-07-01,0.5,0.01,0.80
5021,212311,POTATOES- BAKING PR KG,2024-08-01,0.5,0.01,0.80


In [180]:
df.drop_duplicates(subset=['item_id', 'description'])

,item_id,description,quote_date,quantile,price,price_true
0,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.1,1.574099,1.574099
427,212311,POTATOES- BAKING PR KG,2004-02-01,0.1,0.530000,0.530000
655,310406,FORTIFIED WINE (70-75CL),1988-02-01,0.1,1.990000,1.990000


transparent
hsl(0, 0%, 88%)
hsl(3, 25%, 80%)
hsl(7, 50%, 72%)
hsl(11, 75%, 64%)
hsl(15, 100%, 57%)
hsl(11, 75%, 64%)
hsl(7, 50%, 72%)
hsl(3, 25%, 80%)
hsl(0, 0%, 88%)


In [114]:
proper_quantiles


,item_id,description,quote_date,quantile,price
0,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.1,1.574099
1,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.2,1.741650
2,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.3,1.851881
3,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.4,1.940066
4,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.5,1.962112
...,...,...,...,...,...
9823,310406,FORTIFIED WINE (70-75CL),2024-09-01,0.5,13.250000
9824,310406,FORTIFIED WINE (70-75CL),2024-09-01,0.6,13.490000
9825,310406,FORTIFIED WINE (70-75CL),2024-09-01,0.7,13.500000
9826,310406,FORTIFIED WINE (70-75CL),2024-09-01,0.8,13.500000


In [113]:
df

,item_id,description,quote_date,quantile,price
0,210905,FRESH/CHILLED CHICKEN PER KG,1989-03-01,0.1,1.574099
1,210905,FRESH/CHILLED CHICKEN PER KG,1989-04-01,0.1,1.521188
2,210905,FRESH/CHILLED CHICKEN PER KG,1989-05-01,0.1,1.543234
3,210905,FRESH/CHILLED CHICKEN PER KG,1989-06-01,0.1,1.604963
4,210905,FRESH/CHILLED CHICKEN PER KG,1989-07-01,0.1,1.521188
...,...,...,...,...,...
9823,310406,FORTIFIED WINE (70-75CL),2024-05-01,0.9,2.500000
9824,310406,FORTIFIED WINE (70-75CL),2024-06-01,0.9,1.500000
9825,310406,FORTIFIED WINE (70-75CL),2024-07-01,0.9,2.010000
9826,310406,FORTIFIED WINE (70-75CL),2024-08-01,0.9,2.010000


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

hsl(0, 0%, 88%)
hsl(3, 25%, 80%)
hsl(7, 50%, 72%)
hsl(11, 75%, 64%)
hsl(15, 100%, 57%)
hsl(11, 75%, 64%)
hsl(7, 50%, 72%)
hsl(3, 25%, 80%)
hsl(0, 0%, 88%)


In [105]:
ramp_down

['hsl(15, 100%, 57%)',
 'hsl(12, 80%, 63%)',
 'hsl(9, 60%, 69%)',
 'hsl(6, 40%, 75%)',
 'hsl(3, 20%, 81%)']

In [106]:
ramp_up

['hsl(0, 0%, 88%)',
 'hsl(3, 20%, 81%)',
 'hsl(6, 40%, 75%)',
 'hsl(9, 60%, 69%)',
 'hsl(12, 80%, 63%)']

In [ ]:
def gen_color_range()

In [37]:
df

,item_id,description,quote_date,quantile,price
0,212311,POTATOES- BAKING PR KG,200402.0,0.1,0.530
1,212311,POTATOES- BAKING PR KG,200402.0,0.2,0.698
2,212311,POTATOES- BAKING PR KG,200402.0,0.3,0.790
3,212311,POTATOES- BAKING PR KG,200402.0,0.4,0.850
4,212311,POTATOES- BAKING PR KG,200402.0,0.5,0.860
...,...,...,...,...,...
2047,212311,POTATOES- BAKING PR KG,202409.0,0.5,0.800
2048,212311,POTATOES- BAKING PR KG,202409.0,0.6,0.850
2049,212311,POTATOES- BAKING PR KG,202409.0,0.7,1.140
2050,212311,POTATOES- BAKING PR KG,202409.0,0.8,1.440


In [ ]:


# Flatten multi-index column names if needed
result.columns = ['_'.join(col).strip('_') for col in result.columns]

# Display result
print(result)

In [24]:

   { **{f"q{int(q*100)}": lambda x, q=q: x.quantile(q) for q in quantiles}}


{'q10': <function __main__.<dictcomp>.<lambda>(x, q=0.1)>,
 'q20': <function __main__.<dictcomp>.<lambda>(x, q=0.2)>,
 'q30': <function __main__.<dictcomp>.<lambda>(x, q=0.3)>,
 'q40': <function __main__.<dictcomp>.<lambda>(x, q=0.4)>,
 'q50': <function __main__.<dictcomp>.<lambda>(x, q=0.5)>,
 'q60': <function __main__.<dictcomp>.<lambda>(x, q=0.6)>,
 'q70': <function __main__.<dictcomp>.<lambda>(x, q=0.7)>,
 'q80': <function __main__.<dictcomp>.<lambda>(x, q=0.8)>,
 'q90': <function __main__.<dictcomp>.<lambda>(x, q=0.9)>}

In [11]:
items_df[items_df.description.str.contains('wine', case=False, na=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
470,310304,WINE (PER GLASS),198802,200301,108862
475,310310,"WINE, PER 175 - 250 ML SERVING",200302,202409,105349
479,310315,BOTTLE OF WINE 70-75CL,199402,202409,137319
486,310406,FORTIFIED WINE (70-75CL),198802,202409,92564
488,310409,OTHER IMPTD WHITE WINE-70-75CL,198802,200401,38031
489,310412,IMPORTED RED WINE-70-75 CL BOT,198802,200401,37858
490,310413,WHITE WINE (3 LITRE BOX),199601,199601,212
491,310414,IMPORTED RED WINE LE PIAT D'OR,199601,199601,228
495,310418,IMPORTED SPARKLING WINE 75CL,200302,200601,4498
496,310419,WHITE WINE- EUROPEAN 75CL,200402,202409,54707


In [4]:
prices_df

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id
0,199108.0,2.0,210101,2.0,0.49000,NaN,210101
1,199007.0,10.0,210101,11.0,0.52000,NaN,210101
2,199007.0,2.0,210101,12.0,0.37000,NaN,210101
3,199811.0,179.0,210101,8.0,0.69000,NaN,210101
4,199108.0,1.0,210101,10.0,0.47000,NaN,210101
...,...,...,...,...,...,...,...
46689338,200704.0,441.0,640406,4.0,55.00000,NaN,640406
46689339,200701.0,731.0,640406,3.0,85.00000,NaN,640406
46689340,201210.0,5074.0,640406,3.0,40.00000,NaN,640406
46689341,200812.0,542.0,640406,2.0,45.00000,NaN,640406
